In [25]:
import sqlite3
import pandas as pd
import re
from unicodedata import normalidef extract_text(target_dir: str) -> pd.DataFrame:
    files: Dict[DirName, list[FilePath]] = {}
    files[target_dir] = glob(f"{target_dir}/**.pdf")

    folder = []
    doc_name = []
    page_num = []
    text = []

    for k, v in files.items():
        for f in v:
            with open(f, "rb") as input:
                try:
                    reader = PdfFileReader(input)
                    print(f"{f} has　{reader.getNumPages()}　pages")
                    for i in range(reader.getNumPages()):
                        folder.append(k.split('/')[-1])
                        doc_name.append(f.split('/')[-1])
                        page_num.append(i+1)
                        page = reader.getPage(i)
                        text.append(page.extractText())
                except Exception as e:
                    print(f"failed: {f}")
                    with open(ERR_FILE, "a") as er:
                        er.write(f"{f}: {e}\n")
                    text.append("")

    df = pd.DataFrame(
        {'folder_name': folder, 'file_name': doc_name, 'page': page_num, 'text': text})
    return dfze

In [4]:
def db_con() -> sqlite3.Connection:
    path = "../ffgcrawl/db.sqlite"
    con: sqlite3.Connection = sqlite3.connect(path)
    con.row_factory = sqlite3.Row
    return con

In [9]:
def query(sql: str) -> pd.DataFrame:
    return pd.read_sql(sql, db_con())

In [13]:
texts = query("select * from texts")

In [17]:
t = texts.content[0]

In [38]:
def clean_text(t: str) -> str:
    return normalize("NFKD", re.sub(r"\s+", "", t))

In [39]:
contents = texts.content.map(clean_text)

In [40]:
with open("texts.txt", "w") as fp:
    for c in contents:
        fp.write(c + "\n")
        fp.write("===========\n")